#                   Improve your daily work with Python
##### a webinar by Rajamandala Indonesia
<img src='Rajamandala-Logo.jpeg' >

##### M. Irsyad Hibatullah - Geophysicist at Pertamina Hulu Mahakam

# Case 02: Seismic Batch Preconditioning
#### Sub-Objective:
##### •What kind of Seismic data type do we have?
##### •How the output will be delivered? What kind of output format?
##### •How to read Seismic
##### •How to do the Preconditioning calculation?



### Get the Seismic File List

In [1]:
import os
import glob

InitialFolder = "C:\\Users\\irsya\\Documents\\Jiunx_Files\\PYTHON_PROJECT\\RajamandalaSession\\RAJAMANDALA_SeismicDAta\\Selected"
# PathList_1 = [y for x in os.walk(InitialFolder) for y in glob(os.path.join(x[0], '*.SEGY'))]
# PathList_2 = [y for x in os.walk(InitialFolder) for y in glob(os.path.join(x[0], '*.SGY'))]



PathList = glob.glob(InitialFolder + '**\\*.SGY', recursive=True)
# PathList_2 = glob.glob(InitialFolder + '**\\*.SGY', recursive=True)
# PathList = PathList_1 + PathList_2
for i in PathList:
    print(i)


C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PROJECT\RajamandalaSession\RAJAMANDALA_SeismicDAta\Selected\312_293_mig16res2.sgy
C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PROJECT\RajamandalaSession\RAJAMANDALA_SeismicDAta\Selected\332_298_mig160001.sgy
C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PROJECT\RajamandalaSession\RAJAMANDALA_SeismicDAta\Selected\343_305_ mig000001_sm333_Band-pass.sgy
C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PROJECT\RajamandalaSession\RAJAMANDALA_SeismicDAta\Selected\420_339_mig160001.sgy
C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PROJECT\RajamandalaSession\RAJAMANDALA_SeismicDAta\Selected\432_348_mig160001.sgy


In [2]:
SizeList = []
ExtList = []
FileNameList = []
FolderLocationList = []
for idx,path_result in enumerate(PathList):
    
    
    SizeList.append(os.path.getsize(path_result)/(1024*1024))
    ExtList.append(path_result.split('.')[-1])

    FileNameTemp = path_result.split('.')[0].split('\\')[-1]
    FileNameList.append(FileNameTemp)
    
    FolderLocationList.append(path_result.split(FileNameTemp)[0])
    
    print ('Loading ---- ' + str((idx + 1)/len(path_result)), end="\r")
print ('Completed --- Found ' + str(len(PathList)) + ' Seismic Data')


Completed --- Found 5 Seismic Data


In [4]:
import pandas as pd
ListSeismic_DB = pd.DataFrame(
    {'PathList': PathList,
     'SizeList': SizeList,
     'ExtList': ExtList,
     'FileNameList': FileNameList,
     'FolderLocationList': FolderLocationList,
    }
)

In [5]:
ListSeismic_DB

,PathList,SizeList,ExtList,FileNameList,FolderLocationList
0,C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PR...,1.258095,sgy,312_293_mig16res2,C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PR...
1,C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PR...,1.075180,sgy,332_298_mig160001,C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PR...
2,C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PR...,2.211975,sgy,343_305_ mig000001_sm333_Band-pass,C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PR...
3,C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PR...,1.938725,sgy,420_339_mig160001,C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PR...
4,C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PR...,1.193855,sgy,432_348_mig160001,C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PR...


In [8]:
ListSeismic_DB['Dest_SeismicPath'] = ListSeismic_DB['PathList']

In [10]:
ListSeismic_DB.to_excel('RAJAMANDALA_SeismicList.xlsx')

In [7]:
import pandas as pd 
import segyio
import matplotlib.pyplot as plt 
import glob
import time
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer

def read_segy(srcpath):
    with segyio.open(srcpath, ignore_geometry = True) as src:

        data = src.trace.raw[:]
    return data

def run_SeismicBalancer(srcpath,dstpath, minmax_range=(-10000, 10000)):
    with segyio.open(srcpath,ignore_geometry=True) as src:

        spec = segyio.spec()
        spec.sorting = src.sorting
        spec.format = src.format
        spec.samples = src.samples

        spec.tracecount = src.tracecount
        with segyio.create(dstpath, spec) as dst:
            dst.text[0] = src.text[0]
            dst.bin = src.bin
            dst.header = src.header
            dst.trace = src.trace
            data = src.trace.raw[:].T
            new_data = SeismicBalancer(data, minmax_range=minmax_range)
            dst.trace = new_data.T

def SeismicBalancer(data, minmax_range=(-10000, 10000)):
    
    Standarized = StandardScaler()
    MinMax = MinMaxScaler(feature_range=minmax_range)

    Standarized.fit(data.reshape((-1,1)))
    Standarized.fit(data)
    quantile_transformer = QuantileTransformer(
        output_distribution='normal',
        n_quantiles=100,
        random_state=0)
    datax = Standarized.transform(data)

    return datax


In [13]:
path_excel_file = "RAJAMANDALA_SeismicList.xlsx"

SeismicSurvey_DB = pd.read_excel(path_excel_file, engine='openpyxl')
SeismicSurvey_DB = SeismicSurvey_DB[SeismicSurvey_DB['PathList'].notna()]

list_error = []
list_error_path = []
list_error_filename = []
list_error_survey = []
list_error_survey_sub_folder = []
start = time.time()

error_df = pd.DataFrame()

for i,SeismicSurver_Row in SeismicSurvey_DB.iterrows():
    SeismicPath = SeismicSurver_Row['PathList']

    if i >=0:
        tic = time.time()
        SeismicName = SeismicPath.split('\\')[-1]

        print(SeismicPath)

        
        try :

            Dest_SeismicPath = SeismicSurver_Row["Dest_SeismicPath"].split(".sgy")[0] + "_Balanced.sgy"

            run_SeismicBalancer(SeismicPath, Dest_SeismicPath)
            
            toc = time.time()
            elapsed_time = toc-tic
            print("Job no."+ str(i) + " \t Balancing " + SeismicName + " is Done ;\t\t elapsed time is " + str(elapsed_time) + " s")


        except Exception as error_msg:
        
            list_error.append(SeismicName)
            list_error_path.append(SeismicPath)
            #list_error_survey.append(SeismicSurvey)
            list_error_survey_sub_folder.append(error_msg)
            error_df['seismic_name'] =  list_error
            error_df['seismic_path'] = list_error_path
            #error_df['seismic_survey'] = list_error_survey
            error_df['ERROR'] = list_error_survey_sub_folder

            path_error_log = path_excel_file.split('.')[0]
            error_df.to_excel((path_error_log + "_error_log" + ".xlsx"))
            print(error_msg)

C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PROJECT\RajamandalaSession\RAJAMANDALA_SeismicDAta\Selected\312_293_mig16res2.sgy
Job no.0 	 Balancing 312_293_mig16res2.sgy is Done ;		 elapsed time is 0.08001184463500977 s
C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PROJECT\RajamandalaSession\RAJAMANDALA_SeismicDAta\Selected\332_298_mig160001.sgy
Job no.1 	 Balancing 332_298_mig160001.sgy is Done ;		 elapsed time is 0.0649871826171875 s
C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PROJECT\RajamandalaSession\RAJAMANDALA_SeismicDAta\Selected\343_305_ mig000001_sm333_Band-pass.sgy
Job no.2 	 Balancing 343_305_ mig000001_sm333_Band-pass.sgy is Done ;		 elapsed time is 0.13900041580200195 s
C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PROJECT\RajamandalaSession\RAJAMANDALA_SeismicDAta\Selected\420_339_mig160001.sgy
Job no.3 	 Balancing 420_339_mig160001.sgy is Done ;		 elapsed time is 0.07300019264221191 s
C:\Users\irsya\Documents\Jiunx_Files\PYTHON_PROJECT\RajamandalaSession\RAJAMANDALA_SeismicD